In [72]:
import torch
import  os
torch.__version__ #2.1.1

'2.1.1'

In [73]:
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
# !pip install torch-scatter -f $scatter_src
# !pip install torch-sparse -f $sparse_src
# !pip install torch-geometric
# !pip install ogb

PyG has 2 base classes for storing and/or transforming graphs into tensors:
- `torch_geometric.datasets` contains common graph datasets
- `torch_geometric.data` provides data handling methods for graphs in tensor form

TUDataset: (from TU Dortmund University) benchmark collection that consists of over 120 datasets from a wide range of domains for supervised learning with graphs, i.e., classification and regression

In [74]:
from torch_geometric.datasets import TUDataset 
root = '.enzymes'
name = 'ENZYMES'

# the enzymes dataset with 600 graphs
pyg_dataset = TUDataset(root, name)

# Q1: #classes and #features in enzymes dataset

In [75]:
def get_num_classes(pyg_dataset):
    return pyg_dataset.num_classes

def get_num_features(pyg_dataset):
    return pyg_dataset.num_node_features

num_classes = get_num_classes(pyg_dataset)
num_features = get_num_features(pyg_dataset)
print("{} dataset has {} classes".format(name, num_classes))
print("{} dataset has {} features".format(name, num_features))

ENZYMES dataset has 6 classes
ENZYMES dataset has 3 features


Each PyG `torch_geometric.datasets` dataset has a list of
graphs (in the form of `torch_geometric.data.Data` objects) that we can index into

In [76]:
def get_graph_class(pyg_dataset, idx):
  # takes a PyG dataset object,
  # an index of a graph within the dataset, and 
  # returns the class/label of the graph (as an integer).

  return pyg_dataset[idx].y.item()

graph_0 = pyg_dataset[0]
print(graph_0) # dimensions of edge_index, x, y
idx = 100
label = get_graph_class(pyg_dataset, idx)
print('Graph with index {} has label {}'.format(idx, label))

Data(edge_index=[2, 168], x=[37, 3], y=[1])
Graph with index 100 has label 4


In [77]:
def get_graph_num_edges(pyg_dataset, idx):
  # We assume the graph is undirected
  # You should not count an edge twice if the graph is undirected.
  return pyg_dataset[idx].num_edges // 2

idx = 200
num_edges = get_graph_num_edges(pyg_dataset, idx)
print('Graph with index {} has {} edges'.format(idx, num_edges))

Graph with index 200 has 53 edges


# Open Graph Benchmark (OGB)

In [78]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(
    name=dataset_name, 
    transform=T.ToSparseTensor())

print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

# Extract the graph
data = dataset[0]
print(data)

The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


In [79]:
def graph_num_features(data):
    # TODO: Implement this function that takes a PyG data object,
    # and returns the number of features in the graph (in integer).

    return data.x.shape[1]

num_features = graph_num_features(data)
print('The graph has {} features'.format(num_features))

The graph has 128 features


## Building a GCN for Noe Property Prediction

In [80]:
import numpy as np

def make_symmetric(matrix):
    """
     In an undirected graph, if there is an edge from node A to node B, 
     there should also be an edge from node B to node A. 
     This property should be reflected in the adjacency matrix, 
     which must be symmetric.
    """
    return np.maximum(matrix, matrix.T)

# Example usage
adj_matrix = np.array([[0, 1, 0],
                       [0, 0, 1],
                       [1, 0, 0]])

symmetric_matrix = make_symmetric(adj_matrix)
print(symmetric_matrix)

[[0 1 1]
 [1 0 1]
 [1 1 0]]


In [81]:
import pandas as pd
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch_geometric.nn import GCNConv
from torch.nn import BatchNorm1d, ModuleList
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

If the graph is heterogeneous:

`.get_idx_split()` returns a dictionary where each key is a string ('train', 'valid', or 'test') and each value is another dictionary. This nested dictionary has node types as keys, and the values are torch.Tensor objects of type torch.long, which contain the indices for that node type.

If the graph is not heterogeneous:

`.get_idx_split()` returns a dictionary where each key is a string ('train', 'valid', or 'test') and each value is a torch.Tensor of type torch.long, containing the indices for the nodes.

In [82]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
data = dataset[0]

# Make the adjacency matrix to symmetric
data.adj_t = data.adj_t.to_symmetric()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Device: cpu


In [83]:
data

Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=2315598])

Please follow the figure below to implement the `forward` function.


![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [84]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, 
                num_layers, dropout, return_embeds=False):
        # Initialize self.convs, self.bns, self.softmax.
        super().__init__()

        self.convs = ModuleList([GCNConv(input_dim, hidden_dim)])
        self.convs.extend([GCNConv(hidden_dim, hidden_dim) for i in range(num_layers - 2)])
        self.convs.extend([GCNConv(hidden_dim, output_dim)])
        
        self.bns = ModuleList([BatchNorm1d(hidden_dim) for i in range(num_layers - 1)])
        
        # In this case, we want to apply the softmax function over the 
        # output feature dimension such that each node's output feature vector 
        # sums to 1 for classification probabilities
        self.softmax = torch.nn.LogSoftmax(dim=1)

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()
    

    def forward(self, x, adj_t):
        # TODO: Implement this function that takes the feature tensor x,
        # edge_index tensor adj_t and returns the output tensor as
        # shown in the figure.

        out = None

        ############# Your code here ############
        ## Note:
        ## 1. Construct the network as showing in the figure
        ## 2. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## More information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Don't forget to set F.dropout training to self.training
        ## 4. If return_embed is True, then skip the last softmax layer
        for i, c in enumerate(self.convs):
            if i != len(self.convs) - 1:
                x = c(x, adj_t)
                x = self.bns[i](x)
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training) # training True/False
            else:
                x = c(x, adj_t)
                if self.return_embeds:
                    return x
                out = self.softmax(x)
        return out

In [85]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    
    output = model(data.x, data.adj_t) # calls forward 
    # squeeze: since data.y has dim [N, 1], squeeze to dim [N]
    loss = loss_fn(output[train_idx], data.y[train_idx].squeeze(1))

    loss.backward()
    optimizer.step()

    return loss.item()

In [56]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):

    model.eval()

    # The output of model on all data
    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True) # get indices of max values along dim

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [66]:
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}

model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [68]:
import copy
# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
            f'Loss: {loss:.4f}, '
            f'Train: {100 * train_acc:.2f}%, '
            f'Valid: {100 * valid_acc:.2f}% '
            f'Test: {100 * test_acc:.2f}%')

/Users/choianna5/miniconda3/envs/travel-agent-lam/lib/python3.11/site-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())


Epoch: 01, Loss: 4.2209, Train: 12.92%, Valid: 24.13% Test: 22.35%
Epoch: 02, Loss: 2.2205, Train: 22.48%, Valid: 26.80% Test: 31.76%
Epoch: 03, Loss: 1.8893, Train: 35.66%, Valid: 33.47% Test: 38.35%
Epoch: 04, Loss: 1.7238, Train: 40.49%, Valid: 34.11% Test: 39.46%
Epoch: 05, Loss: 1.6095, Train: 39.55%, Valid: 32.33% Test: 36.20%
Epoch: 06, Loss: 1.5392, Train: 38.19%, Valid: 31.75% Test: 35.29%
Epoch: 07, Loss: 1.4715, Train: 38.07%, Valid: 29.36% Test: 30.31%
Epoch: 08, Loss: 1.4316, Train: 41.23%, Valid: 37.17% Test: 41.56%
Epoch: 09, Loss: 1.3913, Train: 41.18%, Valid: 36.75% Test: 41.23%
Epoch: 10, Loss: 1.3583, Train: 41.33%, Valid: 37.80% Test: 42.05%
Epoch: 11, Loss: 1.3311, Train: 43.13%, Valid: 42.12% Test: 45.51%
Epoch: 12, Loss: 1.3062, Train: 45.10%, Valid: 44.72% Test: 47.59%
Epoch: 13, Loss: 1.2877, Train: 46.49%, Valid: 45.96% Test: 49.10%
Epoch: 14, Loss: 1.2651, Train: 47.22%, Valid: 46.28% Test: 49.70%
Epoch: 15, Loss: 1.2437, Train: 48.51%, Valid: 47.72% Test: 51

In [71]:
best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
    f'Train: {100 * train_acc:.2f}%, '
    f'Valid: {100 * valid_acc:.2f}% '
    f'Test: {100 * test_acc:.2f}%')

Saving Model Predictions
Best model: Train: 72.45%, Valid: 72.62% Test: 72.16%


# GNN: Graph Property Prediction

In [86]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

split_idx = dataset.get_idx_split()

print(f'Task type: {dataset.task_type}')

Device: cpu
Task type: binary classification


In [88]:
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}

In [87]:
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx['valid']], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx['test']], batch_size=32, shuffle=False, num_workers=0)

/Users/choianna5/miniconda3/envs/travel-agent-lam/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


The dataloaders above take our datasets of multiple graphs (`data` objects) and splits them into `torch_geometric.data.Batch` objects. Each `Batch` contains a set of nodes, possibly from different graphs.

`Batch` has these attributes:

- `batch.x`: Node feature matrix of shape `[total_num_nodes, num_node_features]`
- `batch.edge_index`: Adj_t matrix with source and target node indices of all edges in the batch. Has shape `[2, total_num_edges (across all graphs)]`
- `batch.y` (optional): Target labels for graphs or nodes, depending on the task. Has shape `[batch_size, *]`, where `*` depends on the label dimensionality.`
- `Batch.batch`: A vector that maps each node in batch.x to its respective graph in the batch.

    `batch = [0, ..., 0, 1, ..., n - 2, n - 1, ..., n - 1]`

In [91]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool

# GCN for predicting graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout, pool):
        super().__init__()

        self.node_ft_encoder = AtomEncoder(hidden_dim)

        # input, output dims both set to hidden_dim
        self.gcn_layers = GCN(hidden_dim, hidden_dim, hidden_dim,
                            num_layers, dropout, return_embeds=True)
        
        # Add pool param for Q7: experiment with other global pooling layers
        self.pool = pool

        self.pred_head = torch.nn.Linear(hidden_dim, output_dim)

    def reset_parameters(self):
        self.gcn_layers.reset_parameters()
        self.pred_head.reset_parameters()

    def forward(self, batched_data):
        # batched_data: type torch_geometric.data.Batch
        # returns: tensor of dim=#graphs in batch, containing
        #           predicted graph property for each graph

        # Extract important attributes of our mini-batch
        x, adj_t, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        encoded_fts = self.node_ft_encoder(x)

        node_embeddings = self.gcn_layers(encoded_fts, adj_t)
        graph_embeddings = self.pool(node_embeddings, batch)
        graph_properties = self.pred_head(graph_embeddings)

        return graph_properties 

In [92]:
def train(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1: # batch only has 1 node
            pass

        # graph index of the last node in the batch is still 0
        elif batch.batch[-1] == 0: # batch has nodes from only 1 graph
            pass

        else:
            optimizer.zero_grad()
            preds = model(batch) # predicted graph properties
            labels = batch.y.type_as(preds) 

            # boolean tensor; False entries <- NaN (not a number) labels
            is_labeled = (batch.y == batch.y) # since NaN ≠ itself
            # ignore NaN labels when computing training loss
            loss = loss_fn(preds[is_labeled], labels[is_labeled])

            loss.backward()
            optimizer.step()

    return loss.item()

In [103]:
def eval(model, device, loader, evaluator, 
         save_model_results=False, save_file=None):
    model.eval()

    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1: # only one node in batch
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            # view() reshapes tensor
            # detatch() creates new tensor that doesn't need gradients.
            # move to cpu for np operations
            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    # y_true is list of tensors. Cat them tgt 
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    if save_model_results:
        print("Saving Model Predictions")

        # Create a pandas df with 2 cols: | y_pred | y_true |
        data = {}
        data['y_pred'] = y_pred.reshape(-1)
        data['y_true'] = y_true.reshape(-1)

        df = pd.DataFrame(data=data)
        df.to_csv('ogbg-molhiv_graph_' + save_file + '.csv', sep=',', index=False)

    return evaluator.eval(input_dict)

In [101]:
model = GCN_Graph(args['hidden_dim'],
                  dataset.num_tasks,
                  args['num_layers'],
                  args['dropout'],
                  global_mean_pool
                  ).to(device)

evaluator = Evaluator(name='ogbg-molhiv')

In [104]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args['epochs']):
    print("Training...")
    loss = train(model, device, train_loader, optimizer, loss_fn)

    print("Evaluating")
    train_result = eval(model, device, train_loader, evaluator)
    val_result = eval(model, device, valid_loader, evaluator)
    test_result = eval(model, device, test_loader, evaluator)

    train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}/{args['epochs']}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.0254, Train: 83.95%, Valid: 78.83% Test: 75.46%


In [105]:
train_auroc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_auroc = eval(best_model, device, valid_loader, evaluator, save_model_results=True, save_file="valid")[dataset.eval_metric]
test_auroc  = eval(best_model, device, test_loader, evaluator, save_model_results=True, save_file="test")[dataset.eval_metric]

print(f'Best model: '
    f'Train: {100 * train_auroc:.2f}%, '
    f'Valid: {100 * valid_auroc:.2f}% '
    f'Test: {100 * test_auroc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Saving Model Predictions


Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Saving Model Predictions
Best model: Train: 83.07%, Valid: 81.40% Test: 75.53%


## Question 7 (Optional): Experiment with the two other global pooling layers in Pytorch Geometric.

### add_pool

In [106]:
model_add = GCN_Graph(args['hidden_dim'],
                  dataset.num_tasks,
                  args['num_layers'],
                  args['dropout'],
                  global_add_pool
                  ).to(device)

TypeError: GCN_Graph.__init__() takes 5 positional arguments but 6 were given

In [ ]:
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args['epochs']):
    print("Training...")
    loss = train(model, device, train_loader, optimizer, loss_fn)

    print("Evaluating")
    train_result = eval(model, device, train_loader, evaluator)
    val_result = eval(model, device, valid_loader, evaluator)
    test_result = eval(model, device, test_loader, evaluator)

    train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}/{args["epochs"]}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

### max_pool

In [ ]:
model = GCN_Graph(args['hidden_dim'],
                  dataset.num_tasks,
                  args['num_layers'],
                  args['dropout'],
                  global_max_pool
                  ).to(device)

In [ ]:
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args['epochs']):
    print("Training...")
    loss = train(model, device, train_loader, optimizer, loss_fn)

    print("Evaluating")
    train_result = eval(model, device, train_loader, evaluator)
    val_result = eval(model, device, valid_loader, evaluator)
    test_result = eval(model, device, test_loader, evaluator)

    train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}/{args["epochs"]}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')